In [2]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [26]:
df_link_data = pd.read_csv('data/link_data.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20000 non-null  int64 
 1   link        20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [28]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

20000


In [10]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_4_link = breadcrumb.find('a', {'level': '4'})
                if level_4_link:
                    level_4_title = level_4_link.text

                    # Use regular expressions to extract the type_estate
                    match = re.search(r"(.+?) tại", level_4_title)
                    type_estate = match.group(1) if match else ""
                    property_details['type_estate'] = type_estate

                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting type_estate and district from {url}: {e}")

        # Extract price per square meter
        try:
            price_per_sqm_span = soup.find('span', class_='ext')
            if price_per_sqm_span:
                property_details['price_per_sqm'] = price_per_sqm_span.text.strip()
        except AttributeError as e:
            print(f"Error extracting price_per_sqm from {url}: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date from {url}: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details from {url}: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

def scrape_property_details(urls):
    scraped_properties = []
    for url in urls:
        print(f"Processing URL: {url}")
        result = extract_property_details(url)
        print(result)  # Print the extracted details for each URL
        scraped_properties.append(result)
    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT)

Processing URL: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-le-trong-tan-phuong-duong-noi-prj-khu-do-thi-geleximco-le-trong-tan/chinh-chu-ban-can-dep-c49-o20-kdt-geximco-80m-4-g-hoan-thien-vip-pr39840842
{'type_estate': 'Nhà biệt thự, liền kề', 'district': 'Hà Đông', 'price_per_sqm': '~173,12 triệu/m²', 'posted_date': '13/05/2024', 'area': '80 m²', 'price': '13,85 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'interior': 'Đầy đủ', 'num_bedrooms': '6 phòng', 'num_bathrooms': '4 phòng', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
Processing URL: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-tran-phu-phuong-mo-lao/cc-chuyen-nhuong-625m-san-kinh-doanh-mat-ha-dong-so-do-lau-dai-23-ty-pr39824404
{'type_estate': 'Shophouse, nhà phố thương mại', 'district': 'Hà Đông', 'price_per_sqm': '~36,8 triệu/m²', 'posted_date': '11/05/2024', 'area': '625 m²', 'price': '23 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'i

In [33]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_4_link = breadcrumb.find('a', {'level': '4'})
                if level_4_link:
                    level_4_title = level_4_link.text

                    # Use regular expressions to extract the type_estate
                    match = re.search(r"(.+?) tại", level_4_title)
                    type_estate = match.group(1) if match else ""
                    property_details['type_estate'] = type_estate

                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting type_estate and district: {e}")

        # Extract price per square meter
        try:
            price_per_sqm_span = soup.find('span', class_='ext')
            if price_per_sqm_span:
                property_details['price_per_sqm'] = price_per_sqm_span.text.strip()
        except AttributeError as e:
            print(f"Error extracting price_per_sqm: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

def scrape_property_details(urls):
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(extract_property_details, url): url for url in urls}
        try:
            for future in as_completed(futures):
                url = futures[future]
                try:
                    result = future.result()
                    print(result)  # Print the extracted details
                    scraped_properties.append(result)
                except Exception as e:
                    print(f"Failed to extract data from {url}: {e}")
        except KeyboardInterrupt:
            for f in futures:
                f.cancel()
            raise

    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT)
print(LIST_PRODUCT)

Breadcrumb not found for URL: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-tran-phu-phuong-mo-lao/cc-chuyen-nhuong-625m-san-kinh-doanh-mat-ha-dong-so-do-lau-dai-23-ty-pr39824404
{'type_estate': '', 'district': '', 'price_per_sqm': '', 'posted_date': '', 'area': '', 'price': '', 'legal_document': '', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
{'type_estate': 'Nhà riêng', 'district': 'Đống Đa', 'price_per_sqm': '~146,25 triệu/m²', 'posted_date': '13/05/2024', 'area': '40 m²', 'price': '5,85 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'interior': '', 'num_bedrooms': '4 phòng', 'num_bathrooms': '3 phòng', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
Breadcrumb not found for URL: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-le-trong-tan-phuong-duong-noi-prj-khu-do-thi-geleximco-le-trong

In [25]:
print(len(LIST_PRODUCT))

21


In [22]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data.csv')

In [15]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")

# Drop the 'link' and 'describe' columns
df.drop(['link', 'describe'], axis=1, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv("data/raw_data.csv", index=False)

KeyError: "['link', 'describe'] not found in axis"

In [ ]:
df = pd.read_csv('data/raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    4000 non-null   int64  
 1   estate_type   3210 non-null   object 
 2   province      3210 non-null   object 
 3   district      3206 non-null   object 
 4   ward          3206 non-null   object 
 5   price         3824 non-null   object 
 6   square        3770 non-null   object 
 7   post_date     4000 non-null   object 
 8   numb_bedroom  3076 non-null   float64
 9   numb_toilet   2864 non-null   float64
 10  numb_floor    3155 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 343.9+ KB
